[IPython Notebook](20_21_plotting.ipynb) |  [Python Script](20_21_plotting.py)

2.0 - 2.1 Migration: Plotting
============================

In PHOEBE 2.1, the plotting logic has been rewritten from scratch within a separate package called [autofig](https://github.com/kecnry/autofig). By doing so, the plotting is now much more flexible and significantly less buggy while also allowing the plotting logic itself to be tested and used outside of PHOEBE.

Version [XXXX of autofig](https://github.com/kecnry/autofig) is included and built within PHOEBE 2.1 as `phoebe.dependencies.autofig`, regardless of whether you have a standalone autofig installed on your machine.

When calling `b.plot()`, PHOEBE loops through the filtered ParameterSet and makes one or multiple calls to autofig by extracting the necessary arrays and setting reasonable defaults. To see what these calls are, make sure to have the logger set to a level of 'INFO'. `b.plot()` then returns the autofig object, as well as the matplotlib figure object (if you set `show=True` or `save=filename`).

Below we mention the changes to syntax that you need to know if migrating from PHOEBE 2.0 to PHOEBE 2.1.

### Linewidth/Markersize

Autofig attempts to unify calls across matplotlib, including linewidth and markersize. It does so by including the size as an addition dimension `s`. This means that you can pass a value (float) for the size and it will apply to both the linewidth and markersize. If you want separate sizes, you will need to make multiple calls, one with linestyle disabled and the other with marker disabled.

The benefit of this is that you can also pass an array (or access a PHOEBE array by passing a twig/qualifier) and autofig will map that array and scale the size per-point.

In PHOEBE 2.0.x: `b.plot(linewidth=4)`


In PHOEBE 2.1.x: `b.plot(s=4)`, or `b.plot(s='teffs')`.

### Facecolor/Edgecolor Options for Meshes


Similar to size, autofig includes color (for markers and lines), facecolor (for mesh plots only) and edgecolor (for mesh plots only) as additional dimensions.  These are called `c`, `fc`, and `ec`, respectively. This means that you can pass a color (as a string, like 'blue') to any of these arguments, an array (with same length as the underlying data), or a twig/qualifier. The other options for colors (map, lim, unit, label) are all prefixed with this single character (e.g., `fcmap='axmhot'`, `eclabel='temperatures'`).

PHOEBE 2.1 will attempt to map those onto the autofig names, but it may be less confusing to just stick to the single letter versions. The order of fallback preference for PHOEBE 2.1 is the single character, then the plural version (facecolors), and finally the singular version (facecolor). However, this mapping is NOT done for the extra options (map, lim, etc).


### Animations

The `b.animate()` or `PS.animate()` methods have been removed.  Instead, to create an animation, pass `animate=True` (along with `show=True` or `save=filename`) to the `b.plot()` call.  In this case, `b.plot()` will accept *either* `times` or `time` to be sent as the time for each frame of the animation.

When animating, `b.plot()` will still return the autofig object, but the second returned object will be the matplotlib animation object instead of the figure.

### Returned Autofig Object

Calling `b.plot()` now returns the autofig object (as well as the matplotlib figure if showing or saving a figure, otherwise the second returned item will be `None`).

In PHOEBE 2.0.x: `axs, artists = b.plot(..., show=True)`

In PHOEBE 2.1.x: `afig, mplfig = b.plot(..., show=True)` or `afig, mplanim = b.plot(..., animate=True, show=True)`.

With the returned autofig object, you can do anything in the autofig docs, including adding your own external data, re-drawing at different times, re-animating, changing options, etc.  With the returned matplotlib object, you can do anything that you normally could to the matplotlib figure.

But be careful: successive calls to `b.plot()` will keep appending to the same autofig figure **until** you call `show()` and/or `save()`. At that point, the next call to `b.plot()` will start over on a blank figure.  So, if you want to make any changes, you'll either need to rerun the same lines of code, or keep track of the autofig object itself and make changes there.